In [1]:
import transformer as tfm
import dataprocess

In [2]:
import math
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

In [4]:
# Hyperparameter 
unit_size=60
predict_size=5
predict_cycle=30
column_tfm=['close_x','close_y','vol_x']

batch_size=60
dropout=0.1
data_size=3
kq_dim=32
v_dim=32
forward_dim = 2048  # FeedForward dimension

n_layers = 6  # number of Encoder and Decoder Layer
n_heads = 8  # number of heads in Multi-Head Attention

In [5]:
data_SH_SZ_300=pd.read_csv('SH_SZ_300.csv')
trans_data=dataprocess.TransformerData(data_SH_SZ_300,unit_size,predict_size,predict_cycle)
trans_data.create_dataset(column_tfm)
trans_data_loder=Data.DataLoader(trans_data, batch_size)

In [6]:
model = tfm.Transformer(dropout,data_size,kq_dim,v_dim,forward_dim,n_layers,n_layers,n_heads,n_heads)

In [7]:
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [8]:
for epoch in range(1000):
    for enc_inputs, dec_inputs, dec_outputs in trans_data_loder:
        enc_inputs = torch.as_tensor(enc_inputs, dtype=torch.float32)
        dec_inputs = torch.as_tensor(dec_inputs, dtype=torch.float32)
        dec_outputs = torch.as_tensor(dec_outputs, dtype=torch.float32)
        
        # outputs: [batch_size * tgt_len, tgt_vocab_size]
        outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs.size(0),enc_inputs, dec_inputs)
        
        loss = criterion(outputs.reshape(dec_outputs.size()), dec_outputs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 50 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))

E:\transformer\transformer.py:55: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen\native\TensorAdvancedIndexing.cpp:570.)
  scores.masked_fill_(mask, -1e9)
D:\Anaconda3\envs\pytorch_cpu\lib\site-packages\torch\autograd\__init__.py:125: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen\native\TensorAdvancedIndexing.cpp:570.)
  Variable._execution_engine.run_backward(


Epoch: 0001 loss = 0.087635
Epoch: 0051 loss = 0.075982
Epoch: 0101 loss = 0.078616
Epoch: 0151 loss = 0.075411
Epoch: 0201 loss = 0.072655
Epoch: 0251 loss = 0.073021
Epoch: 0301 loss = 0.072859
Epoch: 0351 loss = 0.072834
Epoch: 0401 loss = 0.072838
Epoch: 0451 loss = 0.072791
Epoch: 0501 loss = 0.072776
Epoch: 0551 loss = 0.072787
Epoch: 0601 loss = 0.072781
Epoch: 0651 loss = 0.072780
Epoch: 0701 loss = 0.072799
Epoch: 0751 loss = 0.072778


KeyboardInterrupt: 

In [14]:
model.parameters

<bound method Module.parameters of Transformer(
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder_layers): ModuleList(
    (0): EncoderLayer(
      (encoder_multi_attention): MultiHeadAttention(
        (matrix_Q): Linear(in_features=3, out_features=256, bias=False)
        (matrix_K): Linear(in_features=3, out_features=256, bias=False)
        (matrix_V): Linear(in_features=3, out_features=256, bias=False)
        (fully_con): Linear(in_features=256, out_features=3, bias=False)
      )
      (feedforward): Sequential(
        (0): Linear(in_features=3, out_features=2048, bias=False)
        (1): ReLU()
        (2): Linear(in_features=2048, out_features=3, bias=False)
      )
    )
    (1): EncoderLayer(
      (encoder_multi_attention): MultiHeadAttention(
        (matrix_Q): Linear(in_features=3, out_features=256, bias=False)
        (matrix_K): Linear(in_features=3, out_features=256, bias=False)
        (matrix_V): Linear(in_features=3,